## 1. Load raw data

In [1]:
import pandas as pd
import numpy as np

pd.set_option('display.max_columns', None)

In [2]:
df_daily_rent = pd.read_csv("../data/raw/daily_rent_detail.csv")
df_weather = pd.read_csv("../data/raw/weather.csv")

C:\Users\Hao Le\AppData\Local\Temp\ipykernel_38448\1505384709.py:1: DtypeWarning: Columns (5,7) have mixed types. Specify dtype option on import or set low_memory=False.
  df_daily_rent = pd.read_csv("../data/raw/daily_rent_detail.csv")


# 2. Data cleaning

In [3]:
df_daily_rent_copy = df_daily_rent.copy()
df_weather_copy = df_weather.copy()

# 3. Data Integration

In [13]:
import pandas as pd

# 1. Trích xuất cột cần thiết
df_weather_needs = df_weather_copy[["datetime", "icon"]].copy()
df_daily_rent_needs = df_daily_rent_copy[["started_at"]].copy()

# 2. TẠO CỘT 'datetime' CHO BẢNG RENT 
# Chuyển đổi 'started_at' sang datetime, sau đó lấy phần ngày (YYYY-MM-DD) để khớp với file weather
df_daily_rent_needs['datetime'] = pd.to_datetime(df_daily_rent_needs['started_at'], format='mixed').dt.strftime('%Y-%m-%d')

# (Tùy chọn) Đảm bảo bảng Weather cũng chuẩn format string YYYY-MM-DD để so khớp chính xác
df_weather_needs['datetime'] = pd.to_datetime(df_weather_needs['datetime']).dt.strftime('%Y-%m-%d')

# 3. Merge (Bây giờ cả 2 bảng đều đã có cột 'datetime')
df_final = pd.merge(df_daily_rent_needs, df_weather_needs, how="left", on="datetime")

# Kiểm tra kết quả
print(df_final.head())

            started_at    datetime               icon
0  2020-05-30 17:25:29  2020-05-30               rain
1  2020-05-09 14:42:04  2020-05-09  partly-cloudy-day
2  2020-05-24 17:27:19  2020-05-24             cloudy
3  2020-05-27 15:29:52  2020-05-27  partly-cloudy-day
4  2020-05-31 14:06:03  2020-05-31  partly-cloudy-day


In [14]:
# Check if missing data after merging
assert df_daily_rent.shape[0] == df_final.shape[0]

In [15]:
# Save to
df_final.to_csv("../data/raw/final.csv", index=False)

# 4. Feature engineering

In [16]:
df_temp = pd.to_datetime(df_final["started_at"], format="mixed")

df_final["time"] = pd.DataFrame(df_temp.dt.strftime("%H:%M"))
df_final["dow"] =pd.DataFrame(df_temp.dt.day_name())

In [17]:
df_final.head()

,started_at,datetime,icon,time,dow
0,2020-05-30 17:25:29,2020-05-30,rain,17:25,Saturday
1,2020-05-09 14:42:04,2020-05-09,partly-cloudy-day,14:42,Saturday
2,2020-05-24 17:27:19,2020-05-24,cloudy,17:27,Sunday
3,2020-05-27 15:29:52,2020-05-27,partly-cloudy-day,15:29,Wednesday
4,2020-05-31 14:06:03,2020-05-31,partly-cloudy-day,14:06,Sunday


In [ ]:
# --- PHÂN LOẠI GIỜ TRONG NGÀY --- #

# 1. CHUẨN HÓA CỘT 'time' VỀ DẠNG DATETIME
df_final['time'] = pd.to_datetime(df_final['time'], format='mixed')

# 2. TẠO CỘT GIỜ (0-23)
df_final['hour'] = df_final['time'].dt.hour

# 3. HÀM PHÂN LOẠI (Cao điểm, Thường, Ngoài giờ)
def phan_loai_gio(h):
    # Cao điểm: 7h-9h (tức 7, 8) VÀ 17h-19h (tức 17, 18)
    if (7 <= h < 9) or (17 <= h < 19):
        return 'Giờ cao điểm'
    
    # Ngoài giờ: Sau 19h tối hoặc Trước 7h sáng
    elif (h >= 19) or (h < 7):
        return 'Ngoài giờ'
    
    # Giờ thường: Các giờ còn lại (9h-16h)
    else:
        return 'Giờ thường'

# 4. ÁP DỤNG VÀO DỮ LIỆU
df_final['group_time'] = df_final['hour'].apply(phan_loai_gio)

# --- KIỂM TRA KẾT QUẢ ---
print("✅ Đã thêm cột 'group_time' thành công!")
print("Thống kê số lượng:")
print(df_final['group_time'].value_counts())

print("\nXem 5 dòng đầu tiên:")
print(df_final[['started_at', 'hour', 'group_time', 'datetime', 'icon', 'time', 'dow']].head())

✅ Đã thêm cột 'group_time' thành công!
Thống kê số lượng:
group_time
Giờ thường      7675666
Giờ cao điểm    4482488
Ngoài giờ       3928518
Name: count, dtype: int64

Xem 5 dòng đầu tiên:
            started_at  hour    group_time    datetime               icon  \
0  2020-05-30 17:25:29    17  Giờ cao điểm  2020-05-30               rain   
1  2020-05-09 14:42:04    14    Giờ thường  2020-05-09  partly-cloudy-day   
2  2020-05-24 17:27:19    17  Giờ cao điểm  2020-05-24             cloudy   
3  2020-05-27 15:29:52    15    Giờ thường  2020-05-27  partly-cloudy-day   
4  2020-05-31 14:06:03    14    Giờ thường  2020-05-31  partly-cloudy-day   

                 time        dow  
0 2025-12-28 17:25:00   Saturday  
1 2025-12-28 14:42:00   Saturday  
2 2025-12-28 17:27:00     Sunday  
3 2025-12-28 15:29:00  Wednesday  
4 2025-12-28 14:06:00     Sunday  


In [ ]:
# --- PHÂN LOẠI NGÀY TRONG TUẦN --- #

# 1. Chuẩn hóa cột 'dow' về chữ thường (lowercase)
# .astype(str) để đảm bảo không bị lỗi nếu có dữ liệu lạ không phải chữ
df_final['dow'] = df_final['dow'].astype(str).str.lower().str.strip()

# 2. Định nghĩa danh sách ngày cuối tuần (Tiếng Anh)
# Bạn kiểm tra kỹ file xem dùng 'saturday' hay 'sat' nhé. Ở đây tôi để tên đầy đủ.
nhom_cuoi_tuan = ['saturday', 'sunday']

# 3. Tạo cột mới 'day_type' (Loại ngày)
# np.where(Điều_kiện, Giá_trị_nếu_Đúng, Giá_trị_nếu_Sai)
df_final['day_type'] = np.where(
    df_final['dow'].isin(nhom_cuoi_tuan), 
    'Ngày cuối tuần', 
    'Ngày thường'
)

# --- KIỂM TRA KẾT QUẢ ---
print("✅ Đã thêm cột 'day_type' thành công!")
print("Thống kê số lượng:")
print(df_final['day_type'].value_counts())

print("\nXem thử 5 dòng đầu:")
print(df_final[['started_at', 'hour', 'group_time', 'datetime', 'icon', 'time', 'dow', 'day_type']].head())

✅ Đã thêm cột 'day_type' thành công!
Thống kê số lượng:
day_type
Ngày thường       11234701
Ngày cuối tuần     4851971
Name: count, dtype: int64

Xem thử 5 dòng đầu:
            started_at  hour    group_time    datetime               icon  \
0  2020-05-30 17:25:29    17  Giờ cao điểm  2020-05-30               rain   
1  2020-05-09 14:42:04    14    Giờ thường  2020-05-09  partly-cloudy-day   
2  2020-05-24 17:27:19    17  Giờ cao điểm  2020-05-24             cloudy   
3  2020-05-27 15:29:52    15    Giờ thường  2020-05-27  partly-cloudy-day   
4  2020-05-31 14:06:03    14    Giờ thường  2020-05-31  partly-cloudy-day   

                 time        dow        day_type  
0 2025-12-28 17:25:00   saturday  Ngày cuối tuần  
1 2025-12-28 14:42:00   saturday  Ngày cuối tuần  
2 2025-12-28 17:27:00     sunday  Ngày cuối tuần  
3 2025-12-28 15:29:00  wednesday     Ngày thường  
4 2025-12-28 14:06:00     sunday  Ngày cuối tuần  
